<a href="https://colab.research.google.com/github/hagabbar/craft_prospect/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Import required scripts to run
import numpy as np
from keras import Sequential
from keras.layers import LSTM, Dense, Embedding, SpatialDropout1D


Using TensorFlow backend.


In [0]:
"""
This is a script which will check for non-random subsequences 
from the quantum random number generator onboard the CubeSat. 
The code will first perform a benchmark suite of tests known 
as Diehard tests. We will then use the results from these 
tests in order to train and compare the results of a 
neural network approach (LSTMs).
"""

# Define important global variables here
tot_num_samples = 10000 # total number of training samples (50/50 split between rand and non-rand)
train_split = int(int(tot_num_samples/2) * 0.9) # use 90% for training, 10% for testing
bitstream_len = 32     # length of observation window of network
batch_size = 8         # batch size (how many samples NN sees during each iteration)
#true_rand_url = "https://raw.githubusercontent.com/hagabbar/craft_prospect/master/key_assurance/true_random_nums.txt"

# Generate a set of pseudo random numbers
pseudo_rand_data_train = np.random.uniform(low=0,high=32,size=(train_split,bitstream_len))
pseudo_rand_data_test = np.random.uniform(low=0,high=32,size=(int(tot_num_samples/2) - train_split,bitstream_len))
pseudo_rand_labels_train = np.ones(train_split)
pseudo_rand_labels_test = np.ones(int(tot_num_samples/2) - train_split)

# Inject noise signal into pseudo random number sequences (sine-wave in binary)
time = np.arange(0, bitstream_len, 1)
#nonrand_seq = [0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1]
#nonrand_seq = np.random.shuffle(nonrand_seq)
nonrand_seq = (np.sin(time) + 1)
noise_data_train = np.random.uniform(low=0,high=32,size=(train_split,bitstream_len))
noise_data_train += nonrand_seq
noise_data_test = np.random.uniform(low=0,high=32,size=(int(tot_num_samples/2) - train_split,bitstream_len))
noise_labels_train = np.ones(train_split)
noise_labels_test = np.ones(int(tot_num_samples/2) - train_split)

# Load in set of true random numbers
#true_rand_data_train = np.loadtxt(true_rand_url)[:train_split,:]
#true_rand_data_test = np.loadtxt(true_rand_url)[train_split:int(tot_num_samples/2),:]
#true_rand_labels_train = np.zeros(train_split)
#true_rand_labels_test = np.zeros(int(tot_num_samples/2) - train_split)

# Combine all data and labels into two arrays (x and y respectively)
X_train = np.concatenate((pseudo_rand_data_train,noise_data_train))
X_test = np.concatenate((pseudo_rand_data_test,noise_data_test))
Y_train = np.concatenate((pseudo_rand_labels_train,noise_labels_train))
Y_test = np.concatenate((pseudo_rand_labels_test,noise_labels_test))

# Randomly shuffle training sets
idx_shuffle = np.random.permutation(int(train_split*2))
X_train = X_train[idx_shuffle,:]
Y_train = Y_train[idx_shuffle]


In [0]:
print(nonrand_seq)

[1.00000000e+00 1.84147098e+00 1.90929743e+00 1.14112001e+00
 2.43197505e-01 4.10757253e-02 7.20584502e-01 1.65698660e+00
 1.98935825e+00 1.41211849e+00 4.55978889e-01 9.79344930e-06
 4.63427082e-01 1.42016704e+00 1.99060736e+00 1.65028784e+00
 7.12096683e-01 3.86025081e-02 2.49012753e-01 1.14987721e+00
 1.91294525e+00 1.83665564e+00 9.91148691e-01 1.53779596e-01
 9.44216380e-02 8.67648250e-01 1.76255845e+00 1.95637593e+00
 1.27090579e+00 3.36366116e-01 1.19683759e-02 5.95962355e-01]


In [0]:
# Set-up LSTM network
embed_dim = 128
lstm_out = 200
batch_size = 32

model = Sequential()
model.add(Embedding(2500, embed_dim,input_length = bitstream_len))
SpatialDropout1D(0.2)
#model.add(LSTM(lstm_out, dropout_U = 0.2, return_sequences=True, dropout_W = 0.2))
#model.add(LSTM(lstm_out, dropout_U = 0.2, return_sequences=True, dropout_W = 0.2))
model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(200, dropout=0.2, recurrent_dropout=0.2)`
  # Remove the CWD from sys.path while we load stuff.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 32, 128)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 200)               263200    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 583,401
Trainable params: 583,401
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# Train LSTM network
model.fit(X_train, Y_train, batch_size =batch_size, epochs = 5,  verbose = 5)

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [0]:
score,acc = model.evaluate(X_test,Y_test,batch_size=batch_size,verbose=2)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

Score: 0.00
Validation Accuracy: 1.00
